In [25]:
import networkx as nx
# 将边索引转换为 scipy 稀疏矩阵
import community as community_louvain  # Louvain community detection
from torch_geometric.datasets import Planetoid, TUDataset
from torch_geometric.utils import to_dense_adj, to_scipy_sparse_matrix, from_scipy_sparse_matrix
import scipy.sparse as sp
import numpy as np
import matplotlib.pyplot as plt
import torch
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import community
from node2vec import Node2Vec
from sklearn.cluster import KMeans
#-------------------------------------------------------
#-------------------------------------------------------



dataset = TUDataset(root='../tmp/Proteins', name='PROTEINS')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[1].to(device)
print(data)
edge_index = data.edge_index.cpu().numpy()
num_nodes = data.num_nodes
G = nx.Graph()#创建networkx的空图
for i in range(num_nodes):
    G.add_node(i, features=data.x[i].cpu().numpy())
# 添加边
for i, j in edge_index.T:
    G.add_edge(i, j)
# 使用Node2Vec生成节点嵌入
node2vec = Node2Vec(G, dimensions=32, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
# 获取节点嵌入表示
embeddings = [model.wv[str(node)] for node in G.nodes()]
k=5
kmeans = KMeans(n_clusters=k, random_state=0)
clusters = kmeans.fit_predict(embeddings)

# 创建节点与社区的映射
node_cluster_map = {node: clusters[i] for i, node in enumerate(G.nodes())}

# 将节点划分转换为社区列表
communities_kmeans = []
for community_id in set(node_cluster_map.values()):
    communities_kmeans.append([node for node in node_cluster_map if node_cluster_map[node] == community_id])

# 计算模块度
modularity_kmeans = community.modularity(G, communities_kmeans)
print(f"Number of clusters: {k}, Modularity: {modularity_kmeans:.4f}")

Data(edge_index=[2, 92], x=[27, 3], y=[1])


Computing transition probabilities: 100%|██████████| 27/27 [00:00<00:00, 11673.66it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:00<00:00, 54.64it/s]

Number of clusters: 5, Modularity: 0.6522


In [9]:
dataset = Planetoid(root='../tmp/Cora', name='Cora')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)
print(data)
edge_index = data.edge_index.cpu().numpy()
num_nodes = data.num_nodes
G = nx.Graph()#创建networkx的空图
for i in range(num_nodes):
    G.add_node(i, features=data.x[i].cpu().numpy())
# 添加边
for i, j in edge_index.T:
    G.add_edge(i, j)
# 使用Node2Vec生成节点嵌入
node2vec = Node2Vec(G, dimensions=32, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
# 获取节点嵌入表示
embeddings = [model.wv[str(node)] for node in G.nodes()]
k=7
kmeans = KMeans(n_clusters=k, random_state=0)
clusters = kmeans.fit_predict(embeddings)

# 创建节点与社区的映射
node_cluster_map = {node: clusters[i] for i, node in enumerate(G.nodes())}

# 将节点划分转换为社区列表
communities_kmeans = []
for community_id in set(node_cluster_map.values()):
    communities_kmeans.append([node for node in node_cluster_map if node_cluster_map[node] == community_id])

# 计算模块度
modularity_kmeans = community.modularity(G, communities_kmeans)
print(f"Number of clusters: {k}, Modularity: {modularity_kmeans:.4f}")

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


Computing transition probabilities: 100%|██████████| 2708/2708 [00:00<00:00, 5652.63it/s]
/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
Generating walks (CPU: 1):   0%|          | 0/50 [00:00<?, ?it/s]/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
Generating walks (CPU: 2):   0%|          | 0/50 [00:00<?, ?it/s]/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
Generating walks (CPU: 3): 

Number of clusters: 7, Modularity: 0.7426


In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import gc


##----------read nodes function-------------------------------------
def read_nodes(file_path, node_type):
    with open(file_path, 'r') as file:
        nodes = [line.strip() for line in file]
    node_data = {}
    for node_str in nodes:
        node_parts = node_str.split(',')
        node_id = node_type + node_parts[0]
        #features = list(map(float, node_parts[1:]))
        features = ''  ##For common community detection algorithm,you don't need any features.
        node_data[node_id] = {'node_type': node_type, 'features': features}
    return node_data


##--------------------------------------------------------------------


##------------read edge function-----------------------------------------------
def read_edges_incremental(file_path, edge_type, node_type_map):
    with open(file_path, 'r') as file:
        for line in file:
            edge_str = line.strip()
            edge_parts = edge_str.split(',')
            node1 = node_type_map[0] + edge_parts[0]
            node2 = node_type_map[1] + edge_parts[1]
            #features = list(map(float, edge_parts[2:]))
            features=''
            node1_type = node_type_map[0]
            node2_type = node_type_map[1]
            yield (node1, node2,
                   {'edge_type': edge_type, 'features': features})


##---------------------------------------------------------------------------------------


print("----start to read node.-----------------------")
app_nodes = read_nodes('telecom-graph//node_app.txt', 'app')
package_nodes = read_nodes('telecom-graph//node_package.txt', 'package')
user_nodes = read_nodes('telecom-graph//node_user.txt', 'user')
cell_nodes = read_nodes('telecom-graph//node_cell.txt', 'cell')
G = nx.Graph()
# 添加 package 节点并输出节点数量
for node_id, data in package_nodes.items():
    G.add_node(node_id, **data)
print(f"After adding package nodes: {len(G.nodes())} nodes")

# 添加 app 节点并输出节点数量
for node_id, data in app_nodes.items():
    G.add_node(node_id, **data)
print(f"After adding app nodes: {len(G.nodes())} nodes")

# 添加 user 节点并输出节点数量
for node_id, data in user_nodes.items():
    G.add_node(node_id, **data)
print(f"After adding user nodes: {len(G.nodes())} nodes")

# 添加 cell 节点并输出节点数量
for node_id, data in cell_nodes.items():
    G.add_node(node_id, **data)
print(f"After adding cell nodes: {len(G.nodes())} nodes")
print("-----all nodes is read already.----------------")

print("----------start to read edge.----------------------")

print("----------1--------------------")
for edge in read_edges_incremental('telecom-graph//edge_user_buy_package.txt', 'buy',
                                   ['user', 'package']):
    G.add_edge(edge[0], edge[1], **edge[2])
# 打印图的信息，检查是否添加了边
print(f"After first addition, number of edges: {G.number_of_edges()}")
# 进行垃圾回收，释放内存
gc.collect()

print("----------2--------------------")
for edge in read_edges_incremental('telecom-graph//edge_user_live_cell.txt', 'live',
                                   ['user', 'cell']):
    G.add_edge(edge[0], edge[1], **edge[2])

# 进行垃圾回收，释放内存
gc.collect()
print(f"After second addition, number of edges: {G.number_of_edges()}")
print("----------3--------------------")
for edge in read_edges_incremental('telecom-graph//edge_user_use_app.txt', 'use',
                                   ['user', 'app']):
    G.add_edge(edge[0], edge[1], **edge[2])

# 进行垃圾回收，释放内存
gc.collect()
print(f"After third addition, number of edges: {G.number_of_edges()}")
print("---------all edge is read already.------------------")

#nx.write_graphml(G, "telecom_network_1.graphml")
#print("Writing done!!")

----start to read node.-----------------------
After adding package nodes: 380 nodes
After adding app nodes: 1380 nodes
After adding user nodes: 101380 nodes
After adding cell nodes: 170380 nodes
-----all nodes is read already.----------------
----------start to read edge.----------------------
----------1--------------------
After first addition, number of edges: 1900000
----------2--------------------
After second addition, number of edges: 8800000
----------3--------------------


After third addition, number of edges: 8900000
---------all edge is read already.------------------


In [ ]:
connected_components = list(nx.connected_components(G))
G = G.subgraph(connected_components[0])
# 使用Node2Vec生成节点嵌入
node2vec = Node2Vec(G, dimensions=4, walk_length=1, num_walks=5, workers=8)
model = node2vec.fit(window=5, min_count=1, batch_words=4)
# 获取节点嵌入表示
embeddings = [model.wv[str(node)] for node in G.nodes()]
k=35
kmeans = KMeans(n_clusters=k, random_state=0)
clusters = kmeans.fit_predict(embeddings)

# 创建节点与社区的映射
node_cluster_map = {node: clusters[i] for i, node in enumerate(G.nodes())}

# 将节点划分转换为社区列表
communities_kmeans = []
for community_id in set(node_cluster_map.values()):
    communities_kmeans.append([node for node in node_cluster_map if node_cluster_map[node] == community_id])

# 计算模块度
modularity_kmeans = community.modularity(G, communities_kmeans)
print(f"Number of clusters: {k}, Modularity: {modularity_kmeans:.4f}")

In [20]:
dataset = Planetoid(root='../tmp/Pubmed', name='Pubmed')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)
print(data)
dge_index = data.edge_index.cpu().numpy()
num_nodes = data.num_nodes
G = nx.Graph()#创建networkx的空图
for i in range(num_nodes):
    G.add_node(i, features=data.x[i].cpu().numpy())
# 添加边
for i, j in edge_index.T:
    G.add_edge(i, j)
# 使用Node2Vec生成节点嵌入
node2vec = Node2Vec(G, dimensions=32, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
# 获取节点嵌入表示
embeddings = [model.wv[str(node)] for node in G.nodes()]
k=3
kmeans = KMeans(n_clusters=k, random_state=0)
clusters = kmeans.fit_predict(embeddings)

# 创建节点与社区的映射
node_cluster_map = {node: clusters[i] for i, node in enumerate(G.nodes())}

# 将节点划分转换为社区列表
communities_kmeans = []
for community_id in set(node_cluster_map.values()):
    communities_kmeans.append([node for node in node_cluster_map if node_cluster_map[node] == community_id])

# 计算模块度
modularity_kmeans = community.modularity(G, communities_kmeans)
print(f"Number of clusters: {k}, Modularity: {modularity_kmeans:.4f}")

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


Computing transition probabilities: 100%|██████████| 19717/19717 [00:00<00:00, 162523.79it/s]
/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
Generating walks (CPU: 1):  28%|██▊       | 14/50 [00:00<00:02, 14.50it/s]/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
Generating walks (CPU: 1):  56%|█████▌    | 28/50 [00:01<00:01, 14.06it/s]/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
Gener

Number of clusters: 3, Modularity: 0.4761


In [16]:
dataset = Planetoid(root='../tmp/Citeseer', name='Citeseer')
print(dataset)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)
print(data)
dge_index = data.edge_index.cpu().numpy()
num_nodes = data.num_nodes
G = nx.Graph()#创建networkx的空图
for i in range(num_nodes):
    G.add_node(i, features=data.x[i].cpu().numpy())
# 添加边
for i, j in edge_index.T:
    G.add_edge(i, j)
# 使用Node2Vec生成节点嵌入
node2vec = Node2Vec(G, dimensions=32, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
# 获取节点嵌入表示
embeddings = [model.wv[str(node)] for node in G.nodes()]
k=7
kmeans = KMeans(n_clusters=k, random_state=0)
clusters = kmeans.fit_predict(embeddings)

# 创建节点与社区的映射
node_cluster_map = {node: clusters[i] for i, node in enumerate(G.nodes())}

# 将节点划分转换为社区列表
communities_kmeans = []
for community_id in set(node_cluster_map.values()):
    communities_kmeans.append([node for node in node_cluster_map if node_cluster_map[node] == community_id])

# 计算模块度
modularity_kmeans = community.modularity(G, communities_kmeans)
print(f"Number of clusters: {k}, Modularity: {modularity_kmeans:.4f}")

Citeseer()
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


Computing transition probabilities: 100%|██████████| 3327/3327 [00:00<00:00, 151045.07it/s]
/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
Generating walks (CPU: 1):  38%|███▊      | 19/50 [00:00<00:00, 37.19it/s]/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
Generating walks (CPU: 1):  54%|█████▍    | 27/50 [00:00<00:00, 38.55it/s]/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
Generat

Number of clusters: 7, Modularity: 0.6562


In [24]:
from torch_geometric.datasets import  Actor
dataset =  Actor(root='../tmp/Actor')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)
print(data)
dge_index = data.edge_index.cpu().numpy()
num_nodes = data.num_nodes
G = nx.Graph()#创建networkx的空图
for i in range(num_nodes):
    G.add_node(i, features=data.x[i].cpu().numpy())
# 添加边
for i, j in edge_index.T:
    G.add_edge(i, j)
# 使用Node2Vec生成节点嵌入
node2vec = Node2Vec(G, dimensions=32, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
# 获取节点嵌入表示
embeddings = [model.wv[str(node)] for node in G.nodes()]
k=3
kmeans = KMeans(n_clusters=k, random_state=0)
clusters = kmeans.fit_predict(embeddings)

# 创建节点与社区的映射
node_cluster_map = {node: clusters[i] for i, node in enumerate(G.nodes())}

# 将节点划分转换为社区列表
communities_kmeans = []
for community_id in set(node_cluster_map.values()):
    communities_kmeans.append([node for node in node_cluster_map if node_cluster_map[node] == community_id])

# 计算模块度
modularity_kmeans = community.modularity(G, communities_kmeans)
print(f"Number of clusters: {k}, Modularity: {modularity_kmeans:.4f}")

Data(x=[7600, 932], edge_index=[2, 30019], y=[7600], train_mask=[7600, 10], val_mask=[7600, 10], test_mask=[7600, 10])


Computing transition probabilities: 100%|██████████| 7600/7600 [00:00<00:00, 160587.15it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:01<00:00, 26.99it/s]

Number of clusters: 3, Modularity: 0.4761
